We planned to constrcut a multi-label Text Classification model for classifying federal banking regulations into labels --Money Laundering, Enforcement, Financial Reporting, Consumer Protection, Risk Management, Operational Integrity, Market Conduct. First step is to load the dataset.
We manually downloaded the dataset from the project document.

In [12]:

import json

file_path = r'C:\Users\joyvi\OneDrive\Desktop\Final Project\ecfr-title-12.jsonl'

with open(file_path, 'r') as file:
    count = 0
    for line in file:
        if count >= 5:
            break
        data = json.loads(line)
        print(data)
        count += 1 #printing the five rows from the dataset


{'text': '', 'meta': {'chapter': ['II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'II', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'III', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VI', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII', 'VII',

To read the jsonl file using python, First we can upload files via both ways. uploading to session storage or uploading jsonl file after mounting to google drive. We choose first option and uploaded jsonl file into session storage.

Then we read it using python 'loads' method from json module. Since the dataset is very long, we loaded only five entries as sample with following code. Now the step is to install libraries required for the appication.

In [1]:
import pandas as pd #loading the libraries
import re

# loading jsonl data
def load_data(filepath):
    df = pd.read_json(filepath, lines=True)

    # removing rows with the null values
    df = df[df['text'].notna()]

    # removing rows where 'text' is empty string
    df = df[df['text'] != '']

    return df

# function to clean text data in the DataFrame for removing irrelevant data
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
    
    return cleaned_text

# main pre-processing function
def preprocess_data(filepath):
    # Load data
    df = load_data(filepath)

    # applying text cleaning to the 'text' column
    df['clean_text'] = df['text'].apply(clean_text)

    return df

# taking file as input for cleaning
filepath = r'C:\Users\joyvi\OneDrive\Desktop\Final Project\ecfr-title-12.jsonl'
processed_df = preprocess_data(filepath)

# saving cleaned and pre-processed dataFrame to a new file
output_filepath = r'C:\Users\joyvi\OneDrive\Desktop\Final Project\ecfr-title-12_cleaned.jsonl'  # Specify your output file name
processed_df.to_json(output_filepath, orient='records', lines=True)

print(f"DataFrame cleaned and saved !!!")


DataFrame cleaned and saved !!!


But the dataset might have missing values and not applicable values along with some irrelevant text in middle. They will degrade the relevancy of search engine. It is best approach to preprocess the data before taking the dataset for model construction or engines.

Hence, we performed preprocessing the data and we cleaned the data by removing the missing values and not applicable. We gave the cleaned data as input to the model.

In [3]:
import json # importing modules
import random

def shuffle_jsonl(input_file, output_file):
    # Read the JSONL data from the input file
    with open(input_file, 'r') as file:
        data = [json.loads(line) for line in file]

    # shuffling the data entries
    random.shuffle(data)

    # storing  the shuffled data to the output file
    with open(output_file, 'w') as file:
        for item in data:
            json.dump(item, file)
            file.write('\n')

# taking cleaned file as input and storing shuffled data entries into a new file
input_filename = r'C:\Users\joyvi\OneDrive\Desktop\Final Project\ecfr-title-12_cleaned.jsonl'
output_filename = r'C:\Users\joyvi\OneDrive\Desktop\Final Project\ecfr-title-12_jumbled.jsonl'
shuffle_jsonl(input_filename, output_filename)


We are shuffling the dataset rows as all the labels so we are going to shuffle all the rows in the dataframe for the dataset. So that we will get all laels equally and consistently.

In [12]:
import prodigy #importing libraries and modules
from prodigy.components.db import connect
from sklearn.model_selection import train_test_split

# setting up connection to the Prodigy database
db = connect()


Now the labelling is done for the 800 rows from the dataset. Now we have the dataset in the prodigy and we need to split the dataset into two parts of datasets: training set and target set. For splitting it inside the python itself. So we are loading the dataset from prodigy.

In [13]:
# loading the annotated labels dataset from prodfigy
dataset_name = 'regulations_annotated_ph1'
examples = db.get_dataset(dataset_name)


c:\Users\joyvi\OneDrive\Desktop\Final Project\venv\lib\site-packages\IPython\core\interactiveshell.py:3490: DeprecationWarning: `Database.get_dataset` is deprecated and will be removed in Prodigy v2. Use `Database.get_dataset_examples` or `Database.iter_dataset_examples` instead. 
  if await self.run_code(code, result, async_=asy):


We are giving dataset name and loading getting all the annotations into examples variable.

In [14]:
# splitting data into train_examples and target_examples in 80:20 ratio
train_examples, target_examples = train_test_split(examples, test_size=0.2, random_state=42)


Now we have splitting into 80:20 ratio into train_examples and test_examples respectively

In [15]:
# creating new datasets in Prodigy
db.add_dataset('regulations_annotated_train')
db.add_dataset('regulations_annotated_target')

# importing examples into the appropriate datasets
db.add_examples(train_examples, ['regulations_annotated_train'])
db.add_examples(target_examples, ['regulations_annotated_target'])

Now we are having two variables with annotated first 640 examples in train_examples and next 160 examples in target_examples. Now we will store those examples in seperate datasets train_examples and target_examples. Since, we are not having any datasets till now. So we are creating two datasets "regulations_annotated_train" and "regulations_annotated_target". Now we are transferring the examples from train_examples and target_examples into regulations_annotated_train and regulations_annotated_target datasets.

In [3]:
import spacy

# Update this path to point to the model's root directory
model_path = r"C:\Users\joyvi\OneDrive\Desktop\Final Project\model\model-best"

# Load the model using spaCy
nlp = spacy.load(model_path)

ack = "model laoded sucessfully"
doc = nlp(ack)

# viewing the text classification labels
print(doc.cats)


{'Money Laundering': 0.12280339002609253, 'Enforcement': 0.13775867223739624, 'Financial Reporting': 0.13091662526130676, 'Consumer Protection': 0.12837634980678558, 'Risk Management': 0.1728515625, 'Operational Integrity': 0.17128393054008484, 'Market Conduct': 0.1360095888376236}


Now we need to load the spacy model for predicting the new unseen regulations. And then we must get predictions. In the above code, I loaded a text classification model for predicting the new values.

In [12]:
# function to predict labels using the loaded model
def predict_labels(texts):
    results = []
    for text in texts:
        # Process the text using the model
        doc = nlp(text)
        # Check that `doc.cats` is a dictionary
        if isinstance(doc.cats, dict):
            results.append((text, doc.cats))
        else:
            results.append((text, {}))  # Return empty dict if `doc.cats` is not available
    return results

new_text = [ #giving new unseen regulation for the model
    "Evidence. Every party shall have the right to present a case or defense by oral and documentary evidence, to submit rebuttal evidence, and to conduct such cross-examination as may be required for a full and true disclosure of the facts. Irrelevant, immaterial or unduly repetitious evidence shall be excluded. , (b) Objections. Objections to the admission or exclusion of evidence shall be in short form, stating the grounds of objection relied upon but no argument thereon shall be permitted, except as ordered, allowed, or requested by the presiding officer. Rulings on such objections and all other matters shall be part of the transcript. Failure to object timely to the admission or exclusion of evidence or to any ruling constitutes a waiver of such objection. , (c) Stipulations. Independently of the orders or rulings issued as provided by 622.7(b), the parties may stipulate as to any relevant matters of fact or the authenticity of any relevant documents. Such stipulations may be received in evidence at the hearing, and when so received shall be binding on the parties with respect to the matters therein stipulated. , (d) Official notice. All matters officially noticed by the presiding officer shall appear on the record. ",
]

# getting predictions
results = predict_labels(new_text)

# printing results
for text, labels in results:
    print(f"Text: {text[:75]}...")  # Print first 75 characters of text for brevity
    print("Predicted Labels and Scores:")
    if labels:
        for label, score in labels.items():
            print(f" - {label}: {score:.4f}")
    else:
        print(" - No labels predicted.")


Text: Evidence. Every party shall have the right to present a case or defense by ...
Predicted Labels and Scores:
 - Money Laundering: 0.0000
 - Enforcement: 0.9705
 - Financial Reporting: 0.0000
 - Consumer Protection: 0.0010
 - Risk Management: 0.0000
 - Operational Integrity: 0.0284
 - Market Conduct: 0.0000


In the above code, we defined a prediction function and  a new_text variable for passing the new values. We have given the values and got the predictions.

In [13]:


new_text = [ #giving new unseen regulation for the model
    "(a) Except as provided in paragraph (b) of this section, the rights and obligations of the Banks, FHFA, the Director, the Office of Finance, the United States and the Federal Reserve Banks with respect to: A Book-entry consolidated obligation or Security Entitlement and the operation of the Book-entry system, as it applies to consolidated obligations; and the rights of any Person, including a Participant, against the Banks, FHFA, the Director, the Office of Finance, the United States and the Federal Reserve Banks with respect to: A Book-entry consolidated obligation or Security Entitlement and the operation of the Book-entry system, as it applies to consolidated obligations; are governed solely by regulations of FHFA, including the regulations of this part 1270, the applicable offering notice, applicable procedures established by the Office of Finance, and Federal Reserve Bank Operating Circulars., (b) A security interest in a Security Entitlement that is in favor of a Federal Reserve Bank from a Participant and that is not recorded on the books of a Federal Reserve Bank pursuant to \u00a7 1270.14(c)(1), is governed by the law (not including the conflict-of-law rules) of the jurisdiction where the head office of the Federal Reserve Bank maintaining the Participant's Securities Account is located. A security interest in a Security Entitlement that is in favor of a Federal Reserve Bank from a Person that is not a Participant, and that is not recorded on the books of a Federal Reserve Bank pursuant to \u00a7 1270.14(c)(1), is governed by the law determined in the manner specified in \u00a7 1270.13., (c) If the jurisdiction specified in the first sentence of paragraph (b) of this section is a State that has not adopted Revised Article 8, then the law specified in the first sentence of paragraph (b) of this section shall be the law of that State as though Revised Article 8 had been adopted by that State."]

# getting predictions
results = predict_labels(new_text)

# printing results
for text, labels in results:
    print(f"Text: {text[:75]}...")  # Print first 75 characters of text for brevity
    print("Predicted Labels and Scores:")
    if labels:
        for label, score in labels.items():
            print(f" - {label}: {score:.4f}")
    else:
        print(" - No labels predicted.")


Text: (a) Except as provided in paragraph (b) of this section, the rights and obl...
Predicted Labels and Scores:
 - Money Laundering: 0.0000
 - Enforcement: 0.0058
 - Financial Reporting: 0.5642
 - Consumer Protection: 0.0000
 - Risk Management: 0.0001
 - Operational Integrity: 0.4300
 - Market Conduct: 0.0000


In [14]:
new_text = [ #giving new unseen regulation for the model
    "(a) Expedited processing for proposed institutions. (1) An application for deposit insurance for a proposed institution which will be a subsidiary of an eligible depository institution as defined in \u00a7 303.2(r) or an eligible holding company will be acknowledged in writing by the FDIC and will receive expedited processing unless the applicant is notified in writing to the contrary and provided with the basis for that decision. An eligible holding company is defined as a bank or thrift holding company that has consolidated assets of at least $150 million or more; a BOPEC rating of at least \u201c2\u201d for bank holding companies or an above average or \u201cA\u201d rating for thrift holding companies; and at least 75 percent of its consolidated depository institution assets comprised of eligible depository institutions. The FDIC may remove an application from expedited processing for any of the reasons set forth in \u00a7 303.11(c)(2). , (2) Under expedited processing, the FDIC will take action on an application within 60 days of receipt of a substantially complete application or 5 days after the expiration of the comment period described in \u00a7 303.23, whichever is later. Final action may be withheld until the FDIC has assurance that permission to organize the proposed institution will be granted by the chartering authority. Notwithstanding paragraph (a)(1) of this section, if the FDIC does not act within the expedited processing period, it does not constitute an automatic or default approval. , (b) Standard processing. For those applications that are not processed pursuant to the expedited procedures, the FDIC will provide the applicant with written notification of the final action when the decision is rendered. "]

# getting predictions
results = predict_labels(new_text)

# printing results
for text, labels in results:
    print(f"Text: {text[:75]}...")  # Print first 75 characters of text for brevity
    print("Predicted Labels and Scores:")
    if labels:
        for label, score in labels.items():
            print(f" - {label}: {score:.4f}")
    else:
        print(" - No labels predicted.")



Text: (a) Expedited processing for proposed institutions. (1) An application for ...
Predicted Labels and Scores:
 - Money Laundering: 0.0000
 - Enforcement: 0.9588
 - Financial Reporting: 0.0262
 - Consumer Protection: 0.0000
 - Risk Management: 0.0005
 - Operational Integrity: 0.0145
 - Market Conduct: 0.0000


In [15]:
new_text = [ #giving new unseen regulation for the model
    "(a) Filing procedures - (1) Where to file. An application by an insured state branch seeking approval to conduct activities not permissible for a federal branch, as required by \u00a7 347.212(a) of this chapter, shall be submitted in writing to the appropriate FDIC office., (2) Content of filing. A complete letter application shall include the following information:, (i) A brief description of the activity, including the manner in which it will be conducted and an estimate of the expected dollar volume associated with the activity; , (ii) An analysis of the impact of the proposed activity on the condition of the United States operations of the foreign bank in general and of the branch in particular, including a copy of the feasibility study, management plan, financial projections, business plan, or similar document concerning the conduct of the activity; , (iii) A resolution by the applicant's board of directors, or evidence of approval by senior management if a resolution is not required pursuant to the applicant's organizational documents, authorizing the filing of the application; , (iv) A statement by the applicant of whether it is in compliance with sections 347.209 and 347.210 of this chapter;, (v) A statement by the applicant that it has complied with all requirements of the Board of Governors concerning applications to conduct the activity in question and the status of each such application, including a copy of the Board of Governors' disposition of such application, if applicable; and , (vi) A statement of why the activity will pose no significant risk to the Deposit Insurance Fund. , (3) Board of Governors application. If the application to the Board of Governors contains the information required by paragraph (a) of this section, the applicant may submit a copy to the FDIC in lieu of a separate letter application. , (4) Additional information. The FDIC may request additional information to complete processing., (b) Divestiture or cessation - (1) Where to file. Divestiture plans necessitated by a change in law or other authority, as required by \u00a7 347.212(e) of this chapter, shall be submitted in writing to the appropriate FDIC office., (2) Content of filing. A complete letter application shall include the following information:, (i) A detailed description of the manner in which the applicant proposes to divest itself of or cease the activity in question; and , (ii) A projected timetable describing how long the divestiture or cessation is expected to take. , (3) Additional information. The FDIC may request additional information to complete processing. "]

# getting predictions
results = predict_labels(new_text)

# printing results
for text, labels in results:
    print(f"Text: {text[:75]}...")  # Print first 75 characters of text for brevity
    print("Predicted Labels and Scores:")
    if labels:
        for label, score in labels.items():
            print(f" - {label}: {score:.4f}")
    else:
        print(" - No labels predicted.")

Text: (a) Filing procedures - (1) Where to file. An application by an insured sta...
Predicted Labels and Scores:
 - Money Laundering: 0.0000
 - Enforcement: 0.1518
 - Financial Reporting: 0.0242
 - Consumer Protection: 0.0000
 - Risk Management: 0.0000
 - Operational Integrity: 0.8240
 - Market Conduct: 0.0000


In [16]:
new_text = [ #giving new unseen regulation for the model
    "(a) General applicability. This subpart applies to any U.S. bank holding company that is identified as a global systemically important BHC., (b) Initial applicability. A global systemically important BHC shall be subject to the requirements of this subpart beginning on the later of:, (1) January 1, 2019; or, (2) 1095 days (three years) after the date on which the company becomes a global systemically important BHC"]

# getting predictions
results = predict_labels(new_text)

# printing results
for text, labels in results:
    print(f"Text: {text[:75]}...")  # Print first 75 characters of text for brevity
    print("Predicted Labels and Scores:")
    if labels:
        for label, score in labels.items():
            print(f" - {label}: {score:.4f}")
    else:
        print(" - No labels predicted.")

Text: (a) General applicability. This subpart applies to any U.S. bank holding co...
Predicted Labels and Scores:
 - Money Laundering: 0.0000
 - Enforcement: 0.0080
 - Financial Reporting: 0.9791
 - Consumer Protection: 0.0032
 - Risk Management: 0.0008
 - Operational Integrity: 0.0087
 - Market Conduct: 0.0001
